In [4]:
import librosa
import subprocess
import pandas as pd
import os
import glob
import wave
import numpy as np
from sklearn import metrics

In [2]:
def get_deepformants(audio_file, begin, end):
    proc = subprocess.check_output('/home/jeevan/dev/anaconda3/envs/pytorch/bin/python formants.py ' + audio_file + ' /home/jeevan/Desktop/DeepFormants/test.csv --begin ' +  str(begin) + ' --end ' + str(end), cwd="/home/jeevan/Desktop/DeepFormants/", shell=True)
    df = pd.read_csv("/home/jeevan/Desktop/DeepFormants/test.csv")
    return df.iloc[0,1:].tolist()

In [3]:
get_deepformants('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim10/ux_291_FTBW0_F_190.wav', 0.35, 1.35)

/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/DeepFormants/extract_features.py:270: RuntimeWarning: invalid value encountered in log10
  mspec = np.log10(peri)


[143.40967433995, 53.674720506779, 956.85167169666, 2468.1261476407]

In [6]:
wav_in_file = wave.Wave_read('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim10/ux_291_FTBW0_F_190.wav')

wav_in_num_samples = wav_in_file.getnframes()
N = wav_in_file.getnframes()
dstr = wav_in_file.readframes(N)
data = np.fromstring(dstr, np.int16)

Error: unknown format: 3

In [2]:
df = pd.read_csv("/home/jeevan/Desktop/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/e_synth-vowels_formant_estimation_vowlim10.csv")

In [3]:
vowels = list(set(df["vowel_name"].tolist()))
genders = list(set(df["sex"].tolist()))

In [6]:
metrics_dict = {
    "vowel_name": [],
    "sex": [],
    "F1_praat_rms": [],
    "F2_praat_rms": [],
    "F3_praat_rms": [],
    "F1_deepformant_rms": [],
    "F2_deepformant_rms": [],
    "F3_deepformant_rms": [],

}

for vowel in vowels:
    for gender in genders:
        f1_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_praat_base"]
        f2_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_praat_base"]
        f3_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_praat_base"]

        f1_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_synthvow_praat"]
        f2_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_synthvow_praat"]
        f3_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_synthvow_praat"]

        f1_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_synthvow_deepformant"]
        f2_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_synthvow_deepformant"]
        f3_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_synthvow_deepformant"]

        metrics_dict["vowel_name"].append(vowel)
        metrics_dict["sex"].append(gender)
        metrics_dict["F1_praat_rms"].append(metrics.mean_squared_error(f1_gt, f1_praat, squared=False))
        metrics_dict["F2_praat_rms"].append(metrics.mean_squared_error(f2_gt, f2_praat, squared=False))
        metrics_dict["F3_praat_rms"].append(metrics.mean_squared_error(f3_gt, f3_praat, squared=False))

        metrics_dict["F1_deepformant_rms"].append(metrics.mean_squared_error(f1_gt, f1_deep, squared=False))
        metrics_dict["F2_deepformant_rms"].append(metrics.mean_squared_error(f2_gt, f2_deep, squared=False))
        metrics_dict["F3_deepformant_rms"].append(metrics.mean_squared_error(f3_gt, f3_deep, squared=False))


df_metrics = pd.DataFrame(metrics_dict)

        



In [7]:
df_metrics.to_csv("/home/jeevan/Desktop/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/e_synth-vowels_metrics.csv", index=False)